## Data Integration Process

The data integration process consisted of combining multiple data sources to enrich the flight delay dataset. First, meteorological data was integrated by matching the WMO station codes and corresponding dates, ensuring each flight was associated with weather conditions from the nearest meteorological station. Next, holiday information was incorporated by mapping flight dates to a predefined list of U.S. holidays, adding a binary indicator. This process ensured that all relevant external features—weather conditions and holiday data—were systematically aligned with the flight records, enhancing the dataset for subsequent predictive analysis.

In [1]:
import pandas as pd

In [2]:
# load original dataset
flight_delay = pd.read_csv('flight_delay.csv', delimiter = ';')

print(flight_delay.head())

   day_of_week        date  dep_time  arr_time  crs_arr_time unique_carrier  \
0            4  03/01/2019      1829      1959          1925             WN   
1            4  03/01/2019      1937      2037          1940             WN   
2            4  03/01/2019      1644      1845          1725             WN   
3            4  03/01/2019      1452      1640          1625             WN   
4            4  03/01/2019      1323      1526          1510             WN   

                  airline  flight_num tail_num  actual_elapsed_time  ...  \
0  Southwest Airlines Co.        3920   N464WN                   90  ...   
1  Southwest Airlines Co.         509   N763SW                  240  ...   
2  Southwest Airlines Co.        1333   N334SW                  121  ...   
3  Southwest Airlines Co.         675   N286WN                  228  ...   
4  Southwest Airlines Co.           4   N674AA                  123  ...   

   taxi_in  taxi_out  cancelled  cancellation_code diverted carrier_

In [3]:
# load oaci dataset
flight_delay_icao = pd.read_csv('oaci_coord.csv', delimiter = ';')

print(flight_delay_icao.head())

   AirportID                                         Name          City  \
0          1                               Goroka Airport        Goroka   
1          2                               Madang Airport        Madang   
2          3                 Mount Hagen Kagamuga Airport   Mount Hagen   
3          4                               Nadzab Airport        Nadzab   
4          5  Port Moresby Jacksons International Airport  Port Moresby   

            Country IATA  ICAO             Latitude      Longituede  Altitude  \
0  Papua New Guinea  GKA  AYGA  -6081689834590000.0  145391998291.0      5282   
1  Papua New Guinea  MAG  AYMD      -520707988739.0  145789001465.0        20   
2  Papua New Guinea  HGU  AYMH  -5826789855957030.0        1,44E+30      5388   
3  Papua New Guinea  LAE  AYNZ           -6569803.0     146725977.0       239   
4  Papua New Guinea  POM  AYPY  -9443380355834960.0        1,47E+30       146   

  Timezone DST    TzDatabaseTimezone     Type       Source  
0

In [4]:
# map iata code with icao code (orig airport)
iata_icao = dict(zip(flight_delay_icao['IATA'], flight_delay_icao['ICAO']))

# add a new column oaci_org_airport to the main dataset
flight_delay['oaci_org_airport'] = flight_delay['origin'].map(iata_icao)

print(flight_delay.head())

   day_of_week        date  dep_time  arr_time  crs_arr_time unique_carrier  \
0            4  03/01/2019      1829      1959          1925             WN   
1            4  03/01/2019      1937      2037          1940             WN   
2            4  03/01/2019      1644      1845          1725             WN   
3            4  03/01/2019      1452      1640          1625             WN   
4            4  03/01/2019      1323      1526          1510             WN   

                  airline  flight_num tail_num  actual_elapsed_time  ...  \
0  Southwest Airlines Co.        3920   N464WN                   90  ...   
1  Southwest Airlines Co.         509   N763SW                  240  ...   
2  Southwest Airlines Co.        1333   N334SW                  121  ...   
3  Southwest Airlines Co.         675   N286WN                  228  ...   
4  Southwest Airlines Co.           4   N674AA                  123  ...   

   taxi_out  cancelled  cancellation_code  diverted carrier_delay  \

In [5]:
# obtain meteostat stations
!pip install meteostat

from meteostat import Stations, Daily
from datetime import datetime

# download all meteostat stations
stations = Stations().fetch()
# stations with oaci codes
icao_stations = stations[stations['icao'].notnull()]
# most relevant columns
icao_stations = icao_stations[['icao', 'latitude', 'longitude', 'elevation', 'wmo']]

icao_stations.to_csv('icao_stations.csv', index=False)

print(icao_stations.head())

       icao  latitude  longitude  elevation    wmo
id                                                
00FAY  CXHD   53.1900  -112.2500      688.0  71227
01001  ENJA   70.9333    -8.6667       10.0  01001
01004  ENAS   78.9167    11.9333        8.0  01004
01008  ENSB   78.2500    15.4667        2.0  01008
01025  ENTC   69.6833    18.9167        1.0  01025


In [6]:
# laod meteo stations dataset
icao_stations = pd.read_csv('icao_stations.csv', delimiter = ',')

print(icao_stations.head())

   icao  latitude  longitude  elevation      wmo
0  CXHD   53.1900  -112.2500      688.0  71227.0
1  ENJA   70.9333    -8.6667       10.0   1001.0
2  ENAS   78.9167    11.9333        8.0   1004.0
3  ENSB   78.2500    15.4667        2.0   1008.0
4  ENTC   69.6833    18.9167        1.0   1025.0


In [7]:
# check for null values in wmo
null_wmo = icao_stations['wmo'].isnull().sum()

print(f"Number of missing wmo: {null_wmo}")

Number of missing wmo: 3150


In [8]:
# obtain the nearest wmo stations based on the icao airport coordinates
from scipy.spatial import cKDTree
import numpy as np

# separate stations with valid WMO values
valid_wmo_stations = icao_stations.dropna(subset=['wmo'])

# build a k-d tree for the valid stations
valid_coords = np.array(valid_wmo_stations[['latitude', 'longitude']])
tree = cKDTree(valid_coords)

# get coordinates for stations with missing WMO
missing_coords = np.array(icao_stations[icao_stations['wmo'].isnull()][['latitude', 'longitude']])

# query the nearest stations
distances, indices = tree.query(missing_coords)

# assign the nearest WMO to the missing rows
missing_indices = icao_stations[icao_stations['wmo'].isnull()].index
icao_stations.loc[missing_indices, 'wmo'] = valid_wmo_stations.iloc[indices]['wmo'].values

# verify the dataset
print("Number of missing WMO values after the process:", icao_stations['wmo'].isnull().sum())

icao_stations.to_csv('icao_stations.csv', index=False)

Number of missing WMO values after the process: 0


In [9]:
# laod meteo stations dataset modified
icao_stations_modified = pd.read_csv('icao_stations_modified.csv', delimiter = ';')

print(icao_stations_modified.head())

   icao  latitude longitude  elevation      wmo
0  CXHD    53.190   -112.25      688.0  71227.0
1  ENJA   709.333   -86.667       10.0   1001.0
2  ENAS   789.167   119.333        8.0   1004.0
3  ENSB    78.250   154.667        2.0   1008.0
4  ENTC   696.833   189.167        1.0   1025.0


In [10]:
# map using oaci code in order to merge both datasets
icao_stations_dict = dict(zip(icao_stations_modified['icao'], icao_stations_modified['wmo']))

# add a new column 'wmo' to the main dataset
flight_delay['wmo'] = flight_delay['oaci_org_airport'].map(icao_stations_dict)

# convert wmo column to integer, while preserving the NaN values
flight_delay['wmo'] = pd.to_numeric(flight_delay['wmo'], errors='coerce').astype('Int64')

print(flight_delay.head())

   day_of_week        date  dep_time  arr_time  crs_arr_time unique_carrier  \
0            4  03/01/2019      1829      1959          1925             WN   
1            4  03/01/2019      1937      2037          1940             WN   
2            4  03/01/2019      1644      1845          1725             WN   
3            4  03/01/2019      1452      1640          1625             WN   
4            4  03/01/2019      1323      1526          1510             WN   

                  airline  flight_num tail_num  actual_elapsed_time  ...  \
0  Southwest Airlines Co.        3920   N464WN                   90  ...   
1  Southwest Airlines Co.         509   N763SW                  240  ...   
2  Southwest Airlines Co.        1333   N334SW                  121  ...   
3  Southwest Airlines Co.         675   N286WN                  228  ...   
4  Southwest Airlines Co.           4   N674AA                  123  ...   

   cancelled  cancellation_code  diverted  carrier_delay weather_del

In [11]:
# check for null values in wmo in the main dataset
null_wmo = flight_delay['wmo'].isnull().sum()

print(f"Number of missing wmo: {null_wmo}")

Number of missing wmo: 171


In [12]:
# list the unique airports with missing wmo values
# filter rows where wmo is null
null_wmo_airports = flight_delay[flight_delay['wmo'].isnull()]

# unique airports with missing wmo
unique_airports_with_null_wmo = null_wmo_airports['oaci_org_airport'].unique()

print("Unique OACI airports with missing wmo:")
for airport in unique_airports_with_null_wmo:
    print(airport)

Unique OACI airports with missing wmo:
KLAW


In [13]:
# assign manually the nearest wmo station to KLAW airport
# coordinates of the KLAW airport
latitude = 34.56771
longitude = -98.41664

# fetch nearby stations based on the coordinates
nearby_stations = Stations().nearby(latitude, longitude).fetch()

# limit the results to the 10 closest stations
nearest_stations = nearby_stations.head(3)

print("Closest wmo stations to the specified coordinates:")
print(nearest_stations[['icao', 'latitude', 'longitude', 'wmo']])

Closest wmo stations to the specified coordinates:
       icao  latitude  longitude    wmo
id                                     
72355  KFSI   34.6000   -98.4000  72355
KDUC0  KDUC   34.4713   -97.9599   <NA>
KFDR0  KFDR   34.3667   -98.9833   <NA>


In [14]:
# assign manually the chosen wmo code to rows with KLAW
flight_delay.loc[flight_delay['oaci_org_airport'] == 'KLAW', 'wmo'] = 72355

print("Updated KLAW station with the manually assigned WMO:")
print(flight_delay[flight_delay['oaci_org_airport'] == 'KLAW'])

Updated KLAW station with the manually assigned WMO:
        day_of_week        date  dep_time  arr_time  crs_arr_time  \
57797             3  02/01/2019      1354      1440          1330   
57798             5  04/01/2019      1334      1439          1330   
57799             3  16/01/2019      1319      1412          1330   
57800             1  21/01/2019      1324      1425          1330   
57801             3  23/01/2019      1251      1400          1330   
...             ...         ...       ...       ...           ...   
464317            3  18/06/2019       633       734           705   
464318            7  29/06/2019       857       953           705   
467780            7  01/06/2019      1105      1143          1120   
467781            3  11/06/2019      1111      1149          1120   
467782            4  19/06/2019      1123      1214          1120   

       unique_carrier                       airline  flight_num tail_num  \
57797              MQ  American Eagle Airl

In [15]:
# get unique wmo stations from the dataset
unique_wmo = flight_delay['wmo'].unique()

# convert to int format
unique_wmo_list = sorted(unique_wmo.astype(int))

# format as a Python list for easy copy-paste
wmo_list = ', '.join(map(str, unique_wmo_list))

# text file result
with open('wmo_list.txt', 'w') as f:
    f.write(f"[{wmo_list}]")

print(f"[{wmo_list}]")

[69015, 70026, 70133, 70200, 70219, 70261, 70273, 70296, 70321, 70326, 70350, 70361, 70364, 70371, 70381, 70386, 70387, 70395, 70454, 70580, 71108, 72034, 72201, 72202, 72203, 72204, 72205, 72206, 72207, 72208, 72209, 72210, 72211, 72214, 72216, 72217, 72218, 72219, 72221, 72222, 72223, 72225, 72226, 72228, 72231, 72234, 72235, 72243, 72244, 72247, 72248, 72250, 72251, 72252, 72253, 72256, 72257, 72258, 72259, 72263, 72265, 72266, 72267, 72268, 72270, 72274, 72278, 72281, 72286, 72288, 72290, 72293, 72295, 72297, 72302, 72306, 72308, 72309, 72310, 72312, 72314, 72315, 72317, 72323, 72324, 72326, 72327, 72332, 72334, 72335, 72340, 72344, 72351, 72353, 72355, 72356, 72363, 72365, 72382, 72384, 72386, 72389, 72391, 72392, 72394, 72399, 72401, 72403, 72405, 72406, 72407, 72408, 72410, 72411, 72414, 72416, 72421, 72422, 72423, 72428, 72429, 72432, 72434, 72438, 72439, 72440, 72446, 72450, 72462, 72466, 72475, 72476, 72483, 72487, 72488, 72492, 72493, 72494, 72497, 72502, 72503, 72505, 72507

In [16]:
# read the formatted wmo list
with open('wmo_list.txt', 'r') as f:
    formatted_wmo_list = f.read().strip()

# convert the formatted string into a list of wmo values
wmo_values = formatted_wmo_list.strip('[]').split(', ')

# count the number of unique wmo values in the list
count_wmo = len(wmo_values)

# Display the count
print(f"Number of unique wmo values in the list: {count_wmo}")

Number of unique wmo values in the list: 248


In [17]:
# fetch meteorological data for wmo stations
# define dates
start = datetime(2019, 1, 1)
end = datetime(2019, 6, 30)

# list of wmo stations
wmo_list = unique_wmo_list

# storage for all meteorological data
meteo_data = pd.DataFrame()

# fetch station information
stations = Stations()
stations_df = stations.fetch()

# handle missing wmo values and convert wno to int type
stations_df = stations_df.dropna(subset=['wmo'])
stations_df['wmo'] = stations_df['wmo'].astype(int)

# iterate through each wmo station
for wmo in wmo_list:
    try:
        # filter station by wmo code
        station = stations_df.loc[stations_df['wmo'] == wmo]

        if not station.empty:
            # get daily data
            station_id = station.index[0]
            data = Daily(station_id, start, end).fetch()

            if not data.empty:
                # add wmo as an additional column
                data['wmo'] = wmo
                data['date'] = data.index
                data.reset_index(drop=True, inplace=True)

                # append the data to the main df
                meteo_data = pd.concat([meteo_data, data], ignore_index=True)
                print(f"Data fetched for station {wmo}.")
            else:
                print(f"No data available for station {wmo} in the specified date range.")
        else:
            print(f"Station wmo {wmo} not found in the station dataset.")
    except Exception as e:
        print(f"Error fetching data for station wmo {wmo}: {e}")

# save the data to a file
if meteo_data.empty:
    print("No meteorological data was fetched.")
else:
    meteo_data.to_csv('meteorological_data.csv', index=False)
    print("Meteorological data saved in 'meteorological_data.csv'.")

Data fetched for station 69015.
Data fetched for station 70026.
Data fetched for station 70133.
Data fetched for station 70200.
Data fetched for station 70219.
Data fetched for station 70261.
Data fetched for station 70273.
Data fetched for station 70296.
Data fetched for station 70321.
Data fetched for station 70326.
Data fetched for station 70350.
Data fetched for station 70361.
Data fetched for station 70364.
Data fetched for station 70371.
Data fetched for station 70381.
Data fetched for station 70386.
Data fetched for station 70387.
Data fetched for station 70395.
Data fetched for station 70454.
Data fetched for station 70580.
Data fetched for station 71108.
Data fetched for station 72034.
Data fetched for station 72201.
Data fetched for station 72202.
Data fetched for station 72203.
Data fetched for station 72204.
Data fetched for station 72205.
Data fetched for station 72206.
Data fetched for station 72207.
Data fetched for station 72208.
Data fetched for station 72209.
Data fet

In [18]:
# load the meteorological data
meteo_data = pd.read_csv('meteorological_data.csv')

print(meteo_data.head())

   tavg  tmin  tmax  prcp  snow   wdir  wspd  wpgt    pres  tsun    wmo  \
0   3.5   0.0   7.2   NaN   NaN  327.0  17.8   NaN  1022.2   NaN  69015   
1   4.5   0.0  10.6   NaN   NaN  309.0  17.0   NaN  1025.0   NaN  69015   
2   4.0  -3.3  12.8   NaN   NaN    NaN   6.0   NaN  1025.2   NaN  69015   
3   4.2  -2.2  12.8   NaN   NaN    NaN   4.8   NaN  1020.4   NaN  69015   
4   6.7   1.7  11.7   NaN   NaN    NaN  11.0   NaN  1015.8   NaN  69015   

         date  
0  2019-01-01  
1  2019-01-02  
2  2019-01-03  
3  2019-01-04  
4  2019-01-05  


In [19]:
# ensure the date format in flight_delay
flight_delay['date'] = pd.to_datetime(flight_delay['date'], dayfirst=True)

# ensure the date format in meteo_data
meteo_data['date'] = pd.to_datetime(meteo_data['date'])

# Merge datasets on wmo and date
flight_delay = pd.merge(
    flight_delay,
    meteo_data,
    on=['wmo', 'date'],
    how='left'
)

print("\nMerged Meteo and Flight Delay Dataset:")
print(flight_delay.head())


Merged Meteo and Flight Delay Dataset:
   day_of_week       date  dep_time  arr_time  crs_arr_time unique_carrier  \
0            4 2019-01-03      1829      1959          1925             WN   
1            4 2019-01-03      1937      2037          1940             WN   
2            4 2019-01-03      1644      1845          1725             WN   
3            4 2019-01-03      1452      1640          1625             WN   
4            4 2019-01-03      1323      1526          1510             WN   

                  airline  flight_num tail_num  actual_elapsed_time  ...  \
0  Southwest Airlines Co.        3920   N464WN                   90  ...   
1  Southwest Airlines Co.         509   N763SW                  240  ...   
2  Southwest Airlines Co.        1333   N334SW                  121  ...   
3  Southwest Airlines Co.         675   N286WN                  228  ...   
4  Southwest Airlines Co.           4   N674AA                  123  ...   

   tavg  tmin  tmax  prcp snow   w

In [20]:
# load holidays dataset
holidays_data = pd.read_csv("holiday_dates.csv")

print(holidays_data.head())

         Date      Holiday    WeekDay  Month  Day  Year
0  2004-07-04  4th of July     Sunday      7    4  2004
1  2005-07-04  4th of July     Monday      7    4  2005
2  2006-07-04  4th of July    Tuesday      7    4  2006
3  2007-07-04  4th of July  Wednesday      7    4  2007
4  2008-07-04  4th of July     Friday      7    4  2008


In [21]:
# merge holiday information wihth flight data
# create a set of holiday dates
holiday_dates = set(holidays_data["Date"])

# add a column indicating if the flight occurs on a holiday
flight_delay["is_holiday"] = flight_delay["date"].isin(holiday_dates).astype(int)

flight_delay.to_csv("flight_delay_with_meteo_holidays.csv", index=False)
print("Updated file saved as 'flight_delay_with_meteo_holidays.csv'.")

Updated file saved as 'flight_delay_with_meteo_holidays.csv'.
